# Checkpoint 1

**Due: Friday, 16 October, 2020 at 5:00pm BST**

### Read This First
1. Use the constants provided in the cell below. Do not use your own constants.

2. Put the code that produces the output for a given task in the cell indicated. You are welcome to add as many cells as you like for imports, function definitions, variables, etc. **Additional cells need to be in the proper order such that your code runs correctly the first time through.**

3. **IMPORTANT!** Before submitting your notebook, clear the output by clicking *Restart & Clear Output* from the *Kernel* menu. If you do not do this, the file size of your notebook will be very large.

## Libraries and Constants
Custom imports and constants should be added to a new cell.

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt
from matplotlib import animation, rc
%matplotlib inline
import numpy as np
from scipy import integrate, optimize
import time

plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 16

# Gravitational constant
gg     = 6.67408e-11 # m^3 s^-1 kg^-2
# Lunar mass
mass   = 7.342e22 # kg
# Lunar radius
radius = 1738000 # m
# 1 day in seconds
day    = 3600*24 # seconds

### Initial positions and velocities at t=0
rs = [1842280, 0] # m
vs = [0, 1634]    # m/s

## Equations of motion for the unperturbed case:

$
\Large
\begin{align}
\frac{d^{2} x}{dt^2} = - \frac{G M x}{(x^2 + y^2)^{3/2}}
\end{align}
$

$
\Large
\begin{align}
\frac{d^{2} y}{dt^2} = - \frac{G M y}{(x^2 + y^2)^{3/2}}
\end{align}
$

# Task 1 (30p)

In the cell below, write a function that computes the trajectory from t = 0 to tmax, where tmax is given as an argument to the function. The function should return two arrays for the x and y positions of the trajectory. Each array should have N points (equally spaced in time), where N is given as an argument to the function. You may create additional cells for defining functions.

In [ ]:
def task1(N, tmax):
    """
    Compute orbital trajectory.
    
    Parameters
    ----------
    N : int
        Number of points in trajectory arrays
    tmax : float
        End time of integration in units of seconds.
        
    Returns
    -------
    x : array
        x positions of the trajectory
    y : array
        y positions of the trajectory
    """
    ts = np.linspace(0, tmax, N)
    trange = (ts[0], ts[-1])
    fi = np.array([rs[0], rs[1], vs[0], vs[1]]) # initial conditions
    sol = integrate.solve_ivp(projectile_motion, trange, fi, t_eval=ts, dense_output=True, method = 'RK45', max_step = 100)
    return sol.y[0], sol.y[1] # returns two arrays, one with the x values and one with the y values

In [ ]:
def projectile_motion(t, f):
    """
    f0 = x  => dx/dt  = vx
    f1 = y  => dy/dt  = vy
    f2 = vx => dvx/dt = -gg * mass * x / (x**2 + y **2)**1.5
    f3 = vy => dvy/dt = -gg * mass * y / (x**2 + y **2)**1.5
    """
    
    vals = np.zeros_like(f)
    vals[0] = f[2]
    vals[1] = f[3]
    vals[2] = -gg * mass * f[0] / (f[0]**2 + f[1]**2)**1.5
    vals[3] = -gg * mass * f[1] / (f[0]**2 + f[1]**2)**1.5

    return vals

In [ ]:
import matplotlib # debug import

## Testing task 1

The cell below will run your function with inputs of tmax = 1 day (in seconds) and some number of points. The assert statements below will test that the returned arrays are the correct size.

In [ ]:
t_max = day
n_points = int(t_max / 100)

x_pos, y_pos = task1(n_points, t_max)

assert x_pos.size == n_points
assert y_pos.size == n_points

# Task 1 continued

In the cell below, create an animation of the spacecraft's trajectory for t = 0 to 24 hours that includes a circle representing the Moon. Each frame of the animation should only show the last few points to avoid overlapping a previous orbit. A successful animation will be worth the full 30 points. Alternatively, create a static plot showing the Moon and the spacecraft's trajectory. This will be worth a maximum of 25 points.

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(-2000000, 2000000)
ax.set_ylim(-2000000, 2000000)
point, = plt.plot(0, 0, 'ro') # the spaceship

plt.gca().set_aspect("equal") # make it square

ax.add_artist(plt.Circle((0, 0), radius)) # the moon

def init():
    point.set_data([], [])
    return (point,)

def animate(i):
    point.set_data(x_pos[i], y_pos[i])
    return (point,)
    
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=200, interval=20, blit=True)
HTML(anim.to_jshtml())

# Task 2 (15p)

Determine the orbital period T. Your solution must be calculated numerically, i.e., not simply using the analytical expression. The obtained value must be within +/-1 s of the correct value.

In the cell below, write a function that returns the orbital period in units of seconds.

In [ ]:
t_max = day * 10 # evaluates up to this time
n_points = int(t_max) # number of points to evaluate for
x_pos, y_pos = task1(n_points, t_max)

def task2():
    xsign = np.sign(x_pos) # checks sign of the x coordinate
    signchange = ((np.roll(xsign, 1) - xsign) != 0).astype(int) # rolling sum to check for sign changes
    changes = np.where(signchange == signchange.max())[0] # positions of changes
    return (changes[-1] - changes[0]) / (len(changes)-1) * 2 # time between changes divided by number of changes

## Testing task 2

The cell below will run your function and compare with the analytic answer. They should agree to within 1 second.

In [ ]:
T_calc = task2()
t1 = time.time()
print (f"Calculated orbital period: {T_calc} seconds.")
t2 = time.time()

mu = gg * mass
T_analytic = 2 * np.pi * np.sqrt((rs[0]**3 * mu**2) / (2*mu - rs[0]*vs[1]**2)**3)

print (f"Difference with correct answer: {T_calc - T_analytic} seconds.")
print (f"Solution calculated in {t2-t1} seconds.")

assert abs(T_calc - T_analytic) <= 1

print ("Hooray!")

In [ ]:
print(T_analytic)

# Task 3 (20p)

Now add a correction that makes the gravitational field non-spherical. The correction rotates with the Moon (one full rotation every T$_{Moon}$ = 27.3 days). How long does it take until the spacecraft hits the Moon? The time must be accurate to +/-1 s. Assume the Moon’s surface is a sphere. The equations of motion now become:

$
\Large
\begin{align}
\frac{d^{2} x}{dt^2} = - \frac{G M x}{(x^2 + y^2)^{3/2}}
- \frac{q\ G M x^\prime}{(x^{^\prime2} + y^{^\prime2})^{3/2}}
\end{align}
$

$
\Large
\begin{align}
\frac{d^{2} y}{dt^2} = - \frac{G M y}{(x^2 + y^2)^{3/2}}
- \frac{q\ G M y^\prime}{(x^{^\prime2} + y^{^\prime2})^{3/2}}
\end{align}
$

$
\Large
\begin{align}
x^\prime = x + 0.8\ R \cos \left( \frac{2 \pi t}{T_{Moon}} \right)
\end{align}
$

$
\Large
\begin{align}
y^\prime = y + 0.8\ R \sin \left( \frac{2 \pi t}{T_{Moon}} \right)
\end{align}
$

where q = 0.00025.

## Task 3 objectives:
1. Compute the time at which the spacecraft hits the Moon.
2. Make a plot of the height of the spacecraft above the Moon's surface as a function of time. Don't forget to label axes and include units.

# Task 3 part 1

In the cell below, create a function that returns the impact time accurate to within 1 second.

In [ ]:
q = 0.00025
tmoon = 27.3 * day 

def task3():
    trange = (0, 1000000) # evaluate in this range
    
    fi = np.array([rs[0], rs[1], vs[0], vs[1]]) # initial conditions
    
    sol = integrate.solve_ivp(spacecraft, trange, fi, dense_output=True, method = 'RK45', max_step = 10, events = (object_lands))
    return float(sol.t_events[0]) # Gets time of crash

def spacecraft(t, f): # new differential equations
    
    vals = np.zeros_like(f)
    vals[0] = f[2]
    vals[1] = f[3]
    xprime = f[0] + 0.8 * radius * np.cos(2 * np.pi * t / tmoon)
    yprime = f[1] + 0.8 * radius * np.sin(2 * np.pi * t / tmoon)
    vals[2] = -(gg * mass * f[0] / (f[0]**2 + f[1]**2)**1.5) - (q * gg * mass * xprime / (xprime**2 + yprime**2)**1.5)
    vals[3] = -(gg * mass * f[1] / (f[0]**2 + f[1]**2)**1.5) - (q * gg * mass * yprime / (xprime**2 + yprime**2)**1.5)

    return vals

def object_lands(t, f):
    return np.linalg.norm([f[0], f[1]]) - radius # checks if object crashes

object_lands.terminal = True

## Testing task 3 part 1

The cell below will run your function and print your answer. This will be tested against the correct answer (not given).

In [ ]:
t1 = time.time()
t_impact = task3()
t2 = time.time()

print (f"Time to impact: {t_impact:.2f} seconds ({t_impact / day:.2f} days).")
print (f"Solution calculated in {t2-t1} seconds.")

## Task 3 part 2

In the cell below, plot of the height of the spacecraft above the Moon's surface as a function of time. Don't forget to label axes and include units.

In [ ]:
def task3part2():
    trange = (0, 1000000) 
    
    fi = np.array([1842280, rs[1], vs[0], np.sqrt(gg * mass / 1842280)]) # initial conditions
    
    sol = integrate.solve_ivp(spacecraft, trange, fi, dense_output=True, method = 'RK45', max_step = 100, events = (object_lands))
    dis = np.sqrt(sol.y[0]**2 + sol.y[1]**2) - radius
    return dis, sol.y[0], sol.y[1]

fig, ax = plt.subplots()
dis, xqs, yqs = task3part2()
plt.xlabel("Time (100s)")
plt.ylabel("Distance above the surface of the moon (m)")
plt.plot(dis)

# Task 4 (10p)

Which coordinate (x or y) of the position of the spacecraft after one revolution (orbital period T from task 2) is more sensitive to small changes in the amplitude of the correction? To answer this, calculate the derivatives of dx/dq and dy/dq at t = T, for q = 0. Write your answer in the cell below, describing how you arrived at it. Place any code that demonstrates your solution in the cell with the function called `task4`.

We use the formal definition of a derivative:
\begin{align}
\frac{dx}{dq} \approx \frac{f(q + dq) - f(q)}{dq}
\end{align}
And evaluate this at $q = 0$.

The one that has the highest absolute derivative is $y$, so it is more sensitive to small changes in the amplitude of the correction.

In [ ]:
def task4():
    def task4part1():
        trange = (0, 7139) 
    
        fi = np.array([rs[0], rs[1], vs[0], vs[1]]) # initial conditions
    
        sol = integrate.solve_ivp(spacecraft, trange, fi, dense_output=True, method = 'RK45', max_step = 1)
        return sol.y[0], sol.y[1]
    
    xqs, yqs = task4part1()
    xs, ys = task1(100000, 100000)
    t = int(T_calc) - 1
    dx_dq = np.abs((xqs[t] - xs[t])/q) # formula for derivative
    dy_dq = np.abs((yqs[t] - ys[t])/q) # formula for derivative
    if dx_dq > dy_dq:
        return "x", dx_dq
    else:
        return "y", dy_dq

task4()

# Task 5 (10p)

The positions of the spacecraft at t=0, t=T/2, and t=T are given in the cell below. Use them to determine the amplitude of the correction q. Note, this is a different value than for the previous tasks.

A comment for those interested in space science: this is a highly simplified and unrealistic version of the task NASA scientists had to carry out to map out the gravity at the Moon's surface using "telemetry" data (positions and velocities) of various spacecrafts orbiting the Moon.

Put your code in the cell that starts with `def task5():`.

Your answer should be within 20% of the correct answer.

In [ ]:
r1 = [1842280.0, 0.0]
r2 = [-1856332.7223839264, -717.5195460640389]
r3 = [1842271.070055315, 3847.378923359429]

In [ ]:
def task5():
    """
    Task 5 solution here.
    """
    sol2 = optimize.minimize(minimfunc, x0 = [0, 1634, 0.00025], tol = 100000) # minimize this function
    return sol2.x[2]

def minimfunc(array):
    
    def spacecraft2(t, f):
    
        vals = np.zeros_like(f)
        vals[0] = f[2]
        vals[1] = f[3]
        xprime = f[0] + 0.8 * radius * np.cos(2 * np.pi * t / tmoon)
        yprime = f[1] + 0.8 * radius * np.sin(2 * np.pi * t / tmoon)
        vals[2] = -(gg * mass * f[0] / (f[0]**2 + f[1]**2)**1.5) - (spacecraftq * gg * mass * xprime / (xprime**2 + yprime**2)**1.5)
        vals[3] = -(gg * mass * f[1] / (f[0]**2 + f[1]**2)**1.5) - (spacecraftq * gg * mass * yprime / (xprime**2 + yprime**2)**1.5)

        return vals
    
    trange1 = (0, T_analytic/2)
    trange2 = (0, T_analytic)
    fi = np.array([rs[0], rs[1], array[0], array[1]]) # initial conditions, can we assume vx = 0 or not?
    spacecraftq = array[2]
    
    sol1 = integrate.solve_ivp(spacecraft2, trange1, fi, dense_output=True, method = 'RK45', max_step = 100)
    sol2 = integrate.solve_ivp(spacecraft2, trange2, fi, dense_output=True, method = 'RK45', max_step = 100)

    r2_calc = np.array([sol1.y[0][-1], sol1.y[1][-1]]) # position at t/2
    r3_calc = [sol2.y[0][-1], sol2.y[1][-1]] # position at t
    x = np.linalg.norm(np.array(r2) - r2_calc) + np.linalg.norm(np.array(r3) - r3_calc) # difference between r2 and r3 and calculated
    return x


## Testing task 5

The cell below will run your function and print your answer. This will be tested against the correct answer (not given). Your answer should be within 20% of the correct answer.

In [ ]:
t1 = time.time()
mystery_q = task5()
t2 = time.time()

print (f"q = {mystery_q}")
print (f"Solution calculated in {t2-t1} seconds.")


# Task 6 (15p)

What is the minimum initial height of a circular orbit such that, for the perturbation from task 3 (q = 0.00025), the spacecraft does not collide with the Moon but remains gravitationally bound to it?

The orbit may still show oscillations as in task 3, but the spacecraft cannot not hit the lunar surface.

Create a function `task6` that returns the minimum height of the circular orbit in units of meters. Explain your approach. To obtain full marks, the answer must be correct to +/-1 km.

The following formula for the velocity of a point mass in circular orbit of radius r, orbiting a spherically symmetric body of mass M, may be useful:

$
\Large
\begin{align}
v_{c} = \sqrt{\frac{G\ M}{r}}.
\end{align}
$

In the cell below, create a function that calculates the minimum height of a stable orbit in units of meters. Your answer should be within 1000 meters of the correct answer.

In [ ]:
def task6():
    minheight = optimize.brentq(calculate_hit, 1878613, 1900000, )
    return minheight - radius

def calculate_hit(h):

    trange = (0, 2500000) 
    
    fi = np.array([h, 0, 0, np.sqrt(gg * mass / h)]) # initial conditions
    
    sol = integrate.solve_ivp(spacecraft, trange, fi, dense_output=True, method = 'RK45', max_step = 200)
    dis = np.sqrt(sol.y[0]**2 + sol.y[1]**2) - radius
    mindis = np.amin(dis)
    return mindis


## Testing task 6

The cell below will run your function and print your answer. This will be tested against the correct answer (not given). Your answer should be within 1000 meters of the correct answer.

In [ ]:
t1 = time.time()
min_height = task6()
t2 = time.time()
print (f"Minimum height of stable orbit: {min_height} m.")
print (f"Solution calculated in {t2-t1} seconds.")
